# Introduction

Today we will demonstrate how you can generate advertizing content for your inventory to boost sales. We'll do this taking advantage of Azure Cosmos DB for Mongo DB vCore's [vector similarity search](https://learn.microsoft.com/azure/cosmos-db/mongodb/vcore/vector-search) functionality. We will use OpenAI embeddings to generate vectors for inventory description which expected to vastly enhance its semantics. The vectors are then stored and indexed in the Mongo vCore Database. During the content generation for the advertisement time we will also vectorize the advertisement topic and find matching inventory itmes. We will then use retrival augmented generation (RAG) by sending to top matches to OpenAI to generate a catchy advertisement.

# Scenario

1. Shoe Retailer who wants to sell more shoes 
2. Wants to run advertisement to capitalize on recent trends
2. Wants to use generate advertisement content using the inventory items that matches the trend

## Azure OpenAI <a class="anchor" id="azureopenai"></a>

Finally, let's setup our Azure OpenAI resource Currently, access to this service is granted only by application. You can apply for access to Azure OpenAI by completing the form at https://aka.ms/oai/access. Once you have access, complete the following steps:

- Create an Azure OpenAI resource following this quickstart: https://learn.microsoft.com/azure/ai-services/openai/how-to/create-resource?pivots=web-portal
- Deploy a `completions` and `embeddings` model 
    - For more information on `completions`, go here: https://learn.microsoft.com/azure/ai-services/openai/how-to/completions
    - For more information on `embeddings`, go here: https://learn.microsoft.com/azure/ai-services/openai/how-to/embeddings
- Copy the endpoint, key, deployment names for (embeddings model, completions model) into the config.json file.

## Create an Azure Cosmos DB for MongoDB vCore resource<a class="anchor" id="cosmosdb"></a>
Let's start by creating an Azure Cosmos DB for MongoDB vCore Resource following this quick start guide: https://learn.microsoft.com/en-us/azure/cosmos-db/mongodb/vcore/quickstart-portal

Then copy the connection details (server, user, pwd) into the environment file.

# Preliminaries <a class="anchor" id="preliminaries"></a>
First, let's start by installing the packages that we'll need later. 

In [1]:
! pip install numpy
! pip install openai==1.2.3
#! pip install openai==0.28
! pip install pymongo
! pip install python-dotenv
! pip install azure-core
! pip install azure-cosmos
! pip install tenacity
! pip install gradio


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Please use the example.env as a template to provide the necessary keys and endpoints in your own .env file.
Make sure to modify the env_name accordingly. 

In [19]:
import json
import time
import openai

from dotenv import dotenv_values
from openai import AzureOpenAI

# specify the name of the .env file name 
#env_name = "adgen.env" # following example.env template change to your own .env file name
env_name = "example.env" # following example.env template change to your own .env file name
config = dotenv_values(env_name)

openai.api_type = config['openai_api_type']
openai.api_key = config['openai_api_key']
openai.api_base = config['openai_api_endpoint']
openai.api_version = config['openai_api_version']

openai.api_type = "azure"
openai.api_key = "XXXXXXXXXXXXXXXXXXXXXXXXX"
openai.api_base = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX/"
openai.api_version = "2024-02-01"
print("api_type=",openai.api_type)
print("api_key=",openai.api_key)
print("endpoint=",openai.api_base)
print("version=",openai.api_version)

client = AzureOpenAI(
    api_key=openai.api_key,
    api_version=openai.api_version,
    azure_endpoint = openai.api_base
)

print("client=",client)

api_type= azure
api_key= XXXXXXXXXXXXXXXXXXXXXXXXXXXX
endpoint= XXXXXXXXXXXXXXXXXXXXXXXXXX
version= 2024-02-01
client= <openai.lib.azure.AzureOpenAI object at 0x0000014E47728D50>


# Create embeddings <a class="anchor" id="loaddata"></a>
Here we'll load a sample dataset containing descriptions of Azure services. Then we'll user Azure OpenAI to create vector embeddings from this data.

In [20]:
from openai import AzureOpenAI
import openai
def generate_embeddings(text):
    '''
    Generate embeddings from string of text.
    This will be used to vectorize data and user input for interactions with Azure OpenAI.
    '''
    try:
      deploymentName = "text-embedding-ada-002"
      #deploymentName = "text-recommendation"
      #response = client.embeddings.create( input=text, model="text-embedding-ada-002")
      '''
      client = AzureOpenAI(
      api_key=openai.api_key,
      api_version=openai.api_version,
      azure_endpoint = openai.api_base
      )
      print(client)
      print(openai.api_key)
      print(openai.api_version)
      print(openai.api_base)
      '''
      response = client.embeddings.create(input=text,model=deploymentName)
      #embeddings = openai.Embedding.create(engine=deploymentName, input=text)
      #print(embeddings)
      print(response)
      embeddings = response.data[0].embedding
      #time.sleep(0.5) # rest period to avoid rate limiting on AOAI for free tier
      return embeddings
    except Exception as e:
      # Handle any exceptions that occurred during the request
      print(f"An error occurred: {e}")
      return None
 

embeddings = generate_embeddings("Shoes for San Francisco summer")
if embeddings is not None:
   print(embeddings)



CreateEmbeddingResponse(data=[Embedding(embedding=[0.013928048312664032, -0.018227633088827133, -0.002117219613865018, -0.028481490910053253, -0.0009161046473309398, 0.006358173675835133, -0.03551717475056648, 0.006937965750694275, 0.019374188035726547, -0.019608711823821068, 0.00758290383964777, -0.00019462134514469653, 0.006964024156332016, -0.008905352093279362, -0.0035178419202566147, -0.018318835645914078, 0.01792796514928341, 0.004856576211750507, 0.028716012835502625, -0.017849789932370186, -0.017458919435739517, 0.004286555573344231, 0.004788173828274012, -0.02371286042034626, -0.00694448035210371, -0.004768630024045706, 0.01878788135945797, -0.011022571474313736, -0.02457277663052082, -0.021706387400627136, 0.027256760746240616, -0.00280124437995255, -0.023139582946896553, -0.0028338171541690826, 0.0006429018685594201, -0.006709957495331764, -0.005426596850156784, 0.0009234335157088935, 0.012117010541260242, -0.010325517505407333, 0.012384106405079365, -0.0068663060665130615, 

# Connect and setup Cosmos DB for MongoDB vCore

## Set up the connection

In [21]:
import pymongo

env_name = "example.env"
config = dotenv_values(env_name)

mongo_conn = config['mongo_vcore_connection_string']
mongo_client = pymongo.MongoClient(mongo_conn)

##  Set up the DB and collection

In [22]:
DATABASE_NAME = "AdgenDatabase"
COLLECTION_NAME = "AdgenCollection"

mongo_client.drop_database(DATABASE_NAME)
db = mongo_client[DATABASE_NAME]
collection = db[COLLECTION_NAME]

if COLLECTION_NAME not in db.list_collection_names():
    # Creates a unsharded collection that uses the DBs shared throughput
    db.create_collection(COLLECTION_NAME)
    print("Created collection '{}'.\n".format(COLLECTION_NAME))
else:
    print("Using collection: '{}'.\n".format(COLLECTION_NAME))

Created collection 'AdgenCollection'.



## Create the vector index

In [23]:
db.command({
  'createIndexes': COLLECTION_NAME,
  'indexes': [
    {
      'name': 'vectorSearchIndex',
      'key': {
        "contentVector": "cosmosSearch"
      },
      'cosmosSearchOptions': {
        'kind': 'vector-ivf',
        'numLists': 1,
        'similarity': 'COS',
        'dimensions': 1536
      }
    }
  ]
});

## Upload data to the collection
A simple `insert_many()` to insert our data in JSON format into the newly created DB and collection.

In [24]:
data_file = open(file="./data/shoes_with_vectors.json", mode="r") 
data = json.load(data_file)
data_file.close()

collection.insert_many(data)

InsertManyResult([ObjectId('6625fec1014b4a3663f87f7a'), ObjectId('6625fec1014b4a3663f87f7b'), ObjectId('6625fec1014b4a3663f87f7c'), ObjectId('6625fec1014b4a3663f87f7d'), ObjectId('6625fec1014b4a3663f87f7e'), ObjectId('6625fec1014b4a3663f87f7f'), ObjectId('6625fec1014b4a3663f87f80'), ObjectId('6625fec1014b4a3663f87f81'), ObjectId('6625fec1014b4a3663f87f82'), ObjectId('6625fec1014b4a3663f87f83'), ObjectId('6625fec1014b4a3663f87f84'), ObjectId('6625fec1014b4a3663f87f85'), ObjectId('6625fec1014b4a3663f87f86'), ObjectId('6625fec1014b4a3663f87f87'), ObjectId('6625fec1014b4a3663f87f88'), ObjectId('6625fec1014b4a3663f87f89'), ObjectId('6625fec1014b4a3663f87f8a'), ObjectId('6625fec1014b4a3663f87f8b'), ObjectId('6625fec1014b4a3663f87f8c'), ObjectId('6625fec1014b4a3663f87f8d'), ObjectId('6625fec1014b4a3663f87f8e'), ObjectId('6625fec1014b4a3663f87f8f'), ObjectId('6625fec1014b4a3663f87f90'), ObjectId('6625fec1014b4a3663f87f91'), ObjectId('6625fec1014b4a3663f87f92'), ObjectId('6625fec1014b4a3663f87f

# Vector Search in Cosmos DB for MongoDB vCore

In [25]:
# Function to assist with vector search
def vector_search(query, num_results=3):
    
    query_vector = generate_embeddings(query)

    embeddings_list = []
    pipeline = [
        {
            '$search': {
                "cosmosSearch": {
                    "vector": query_vector,
                    "numLists": 1,
                    "path": "contentVector",
                    "k": num_results
                },
                "returnStoredSource": True }},
        {'$project': { 'similarityScore': { '$meta': 'searchScore' }, 'document' : '$$ROOT' } }
    ]
    results = collection.aggregate(pipeline)
    return results

## Perform vector search query

In [27]:
query = "Shoes for Seattle sweater weather"
results = vector_search(query, 3)

print("\nResults:\n")
for result in results: 
    print(result)
    print(f"Similarity Score: {result['similarityScore']}")  
    print(f"Title: {result['document']['name']}")  
    print(f"Price: {result['document']['price']}")  
    print(f"Material: {result['document']['material']}") 
    print(f"Image: {result['document']['img_url']}\n") 
    #print(f"purchase: {result['document']['purchase_url']}\n")

CreateEmbeddingResponse(data=[Embedding(embedding=[0.011651516892015934, -0.020172009244561195, -0.005559493321925402, -0.019222436472773552, -0.01238294504582882, 0.014423243701457977, -0.015488305129110813, -0.011272971518337727, 0.014487404376268387, -0.034543924033641815, -0.0010474175214767456, -0.004176837857812643, 0.004747864790260792, -0.01030415017157793, -0.007397686596959829, -0.029975708574056625, 0.02761460840702057, 0.012055727653205395, 0.02158353663980961, -0.03259345144033432, -0.015950258821249008, 0.013589159585535526, 0.029128793627023697, -0.01301813218742609, -0.005918791051954031, -0.021827345713973045, 0.01979988068342209, -0.031027939170598984, -0.003054032102227211, -0.013152869418263435, 0.028256211429834366, -0.012344448827207088, -0.02848718874156475, -0.015180335380136967, -0.0076414961367845535, -0.015424145385622978, -0.0023161880671977997, 0.007307862397283316, -0.011118986643850803, -0.0178879015147686, 0.009136431850492954, -0.0140767777338624, -0.00

# Generating Ad content with GPT-4 and DALL.E 3

Finally, we put it all together by creating an ad caption and an ad image via the `Completions` API and `DALL.E 3` API, and combine that with the vector search results.

In [34]:
from openai import OpenAI

def generate_ad_title(ad_topic):
    system_prompt = '''
    You are an intelligent assistant for generating witty and cativating tagline for online advertisement.
        - The ad campaign taglines that you generate are short and typically under 100 characters.
    '''

    user_prompt = f'''Generate a catchy, witty, and short sentence (less than 100 characters) 
                    for an advertisement for selling shoes for {ad_topic}'''
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]

    response = client.chat.completions.create(
        model="gpt-35-turbo",
        messages=messages
    )
    
    return response.choices[0].message.content
    print("Here1")
def Generate_ad_image(ad_topic):
    daliClient = OpenAI(
        #api_key=config['dali_api_key']
        api_key="XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
    )

    image_prompt = f'''
        Generate a photorealistic image of an ad campaign for selling {ad_topic}. 
        The image should be clean, with the item being sold in the foreground with an easily identifiable landmark of the city in the background.
        The image should also try to depict the weather of the location for the time of the year mentioned.
        The image should not have any generated text overlay.
    '''

    response = daliClient.images.generate(
        model="dall-e-3",
        prompt= image_prompt,
        size="1024x1024",
        quality="standard",
        n=1,
        )

    return response.data[0].url
    print("Here2")
def render_html_page(ad_topic):

    # Find the matching shoes from the inventory
    results = vector_search(ad_topic, 4)

    ad_header = generate_ad_title(ad_topic)
    ad_image_url = Generate_ad_image(ad_topic)


    with open('./data/ad-start.html', 'r', encoding='utf-8') as html_file:
        html_content = html_file.read()

    html_content += f'''<header>
            <h2>{ad_header}</h1>
        </header>'''    

    html_content += f'''
            <section class="ad">
            <img src="{ad_image_url}" alt="Base Ad Image" class="ad-image">
        </section>'''

    for result in results: 
        html_content += f''' 
        <section class="product">
            <img src="{result['document']['img_url']}" alt="{result['document']['name']}" class="product-image">
            <div class="product-details">
                <h3 class="product-title" color="gray">{result['document']['name']}</h2>
                <p class="product-price">{"$"+str(result['document']['price'])}</p>
                <p class="product-description">{result['document']['description']}</p>
                <a href="#" class="buy-now-button">Buy Now</a>
            </div>
        </section>
        '''

    html_content += '''</article>
                    </body>
                    </html>'''

    return html_content

# Putting it all together

In [35]:
import gradio as gr

css = """
    button { background-color: purple; color: read; }
    <style>
    </style>
"""

with gr.Blocks(css=css, theme=gr.themes.Default(spacing_size=gr.themes.sizes.spacing_sm, radius_size="none")) as demo:
    subject = gr.Textbox(placeholder="subject", label="Ad keywords")
    btn = gr.Button("Generate Ad")
    output_html = gr.HTML(label="Generated Ad HTML")

    btn.click(render_html_page, [subject], output_html)

    btn = gr.Button("Copy HTML")

if __name__ == "__main__":
    demo.launch()   

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


CreateEmbeddingResponse(data=[Embedding(embedding=[0.011651516892015934, -0.020172009244561195, -0.005559493321925402, -0.019222436472773552, -0.01238294504582882, 0.014423243701457977, -0.015488305129110813, -0.011272971518337727, 0.014487404376268387, -0.034543924033641815, -0.0010474175214767456, -0.004176837857812643, 0.004747864790260792, -0.01030415017157793, -0.007397686596959829, -0.029975708574056625, 0.02761460840702057, 0.012055727653205395, 0.02158353663980961, -0.03259345144033432, -0.015950258821249008, 0.013589159585535526, 0.029128793627023697, -0.01301813218742609, -0.005918791051954031, -0.021827345713973045, 0.01979988068342209, -0.031027939170598984, -0.003054032102227211, -0.013152869418263435, 0.028256211429834366, -0.012344448827207088, -0.02848718874156475, -0.015180335380136967, -0.0076414961367845535, -0.015424145385622978, -0.0023161880671977997, 0.007307862397283316, -0.011118986643850803, -0.0178879015147686, 0.009136431850492954, -0.0140767777338624, -0.00